# Practical session 3: Parallel behaviors and more sensing abilities

In the last practical session, we saw how to define, attach, start, stop and detach a behavior on an E-Puck robot. We have implemented three distinct behaviors: `slow_down`, `fear` and `aggression`.

In this section we will see more sensing abilities the E-Puck is equipped with, will define new behaviors using them and will see how to deal with multiple behaviors running in parallel on the same robot. At the end of the session, we will rapidly see how to attach those behaviors on multiple robots interacting together within a V-REP scene (this part will be more properly studied in a next session).

Open V-REP and load the scene `epuck-scene-3.tt` located in the directpry `rti2016/pyvrep_epuck/vrep-scenes`.

**Preliminary notes:** I have added a function to properly close all the processes running in the notebook (robots, behaviors and connection with the simulator). This should significantly limit the amount of connection errors you will encounter. Now the workflow goes like this:

Import the functions `open_session` (equivalent to the function `get_session` we were using before) and `close_session`. Then, obtain a reference to the simulator and the E-Puck by calling `open_session`:

In [1]:
from simulator_interface import open_session, close_session
simulator, epuck = open_session()

Then execute the code you will encounter in the notebook or the code you will write for answering the questions. Whenever you want to restart from scratch (e.g. because something goes wrong), first close the session by executing:

In [10]:
close_session(simulator, epuck)

This will properly close all the running processes. Then restart the notebook (`Kernel -> Restart`) and open the session again:

In [3]:
from simulator_interface import open_session, close_session
simulator, epuck = open_session()

And you will restart with a clean and fresh session.

## Selectively detecting scene objects

To define a repertoire of interesting behaviors, we need the robot able to selectively sense the proximity of different types of objects. For example, one might want to define a behavior for obstacle avoidance and another one for attraction towards mates. The first behavior will require the proximity from walls and pillars, whereas the second will require the proximity from other robots. 

We can filter the result returned by the E-Puck's proximeters by providing the argument `tracked_objects` to the `prox_activations` function:

In [ ]:
left, right = epuck.prox_activations(tracked_objects=["Cup"])
print left, right

Executing the cell above will return the proximeter activations only for the `Cup` objects (the kind of trash bins in the V-REP scene). Give it a try by moving a cup in the detection area of the proximiters and re-executing the cell above to observe the change in the returned values.

The `tracked_objects` argument requires a list of strings (`["Cup"]` in the example above). In Python, a list is a collection of values separated by commas and surrounded by square bracket: `["Cup"]` is therefore a list of only one element (the string `"Cup"`), whereas `["Cup", "ePuck"]` is a list of two elements (the strings `"Cup"` and `"ePuck"`). 
The `tracked_objects` argument, as its name indicates, sets the objects to be tracked by the proximeters. Each object in a V-REP scene has a name, which is shown when you select an object by clicking on it in the interface. One can also inspect the names of all the objects in the `Scene hierarchy` panel on the left (if not visible, you can activate in the `Tools` menu). For example, we see that the cups have the names `Cup`, `Cup0`, `Cup1` etc ..

In the cell above, `tracked_objects=["Cup"]` means *only returns the proximeter activations of objects having their names starting with `"Cup"`*. Since only cups have their names starting with `Cup`, it will return the proximeter activation only for cups, not considering e.g. walls and pillars.

**Q1:** Write the code printing the proximeter activations for walls and pillars and test that it works as expected by placing your epuck close to that objects and verifying the returned values:

In order to track several types of objects at the same time, we can pass several strings to `tracked_objects`. For example, if we want to track both cups and trees, we will write:

In [ ]:
left, right = epuck.prox_activations(tracked_objects=["Cup", "Tree"])
print left, right

This is because all trees in the scene have their name starting with `Tree` (`Tree`, `Tree#0`, `Tree#1` etc..., as shown in the `Scene hierarchy` panel).

**Q2:** Define an `obstacle_avoidance` behavior. Obstacles are walls, pillars and trees, but not cups.  The robot has to turn in the direction opposite to the obstacle, with its speed inversely proportional to the proximeter activations (the closer an obstacle, the slower the speed). 
*Tip:* although quite paradoxal, it is actually very similar to one of the [`love` behaviors of Braitenberg vehicles](https://cdn.rawgit.com/clement-moulin-frier/rti_course/master/class_1/intro.sozi.html#frame1550).

In [ ]:
def obstacle_avoidance(epuck):
    # Write your code below
    

Remember that to test a behavior, you first have to detach all behaviors that could still be attached to the E-Puck, then to attach and start the new one, that is:

In [ ]:
epuck.detach_all_behaviors()
epuck.attach_behavior(obstacle_avoidance)
epuck.start_all_behaviors()

## Environmental dynamics

Until now the environment in which the E-Puck is evolving is quite static: although some objects can be pushed by the robot (e.g. the cups), there is nothing that appears or disappears in the environment. We are now going to see how we can generate food sources appearing at random positions in the environment and disappearing whenever a robot eat them. A food source is modeled as V-REP `Sphere` object, meaning that it can roll on the floor. Making such spheres to appear at regular time intervals and at random positions in the environment is done with:

In [5]:
simulator.start_sphere_apparition(period=5.)

where `period` indicates the time interval at which spheres will appear (here every 5s). In order to stop sphere apparition:

In [7]:
simulator.stop_sphere_apparition()

We can also specify in which area spheres appear through the `min_pos` and `max_pos` arguments:

In [ ]:
simulator.start_sphere_apparition(period=5, min_pos=[-2, -1, 1], max_pos=[2., 0.5, 2])

The cell above will generate spheres at random 3D positions $(x, y, z)$ in the scene with $x\in[-2, 2]$, $x\in[-1, 0.5]$ and $z\in[1, 2]$ (analyze the cell above to understand how the `min_pos` and `max_pos` arguments are converted in $(x, y, z)$ intervals and ask me if it is not clear). You can check how the $x, y, z$ axes are oriented at the bottom-left corner of the V-REP scene (try to rotate the scene while looking at it). When selecting an object, you can check the coordinate of its center in the text located in the top-left corner of the scene. The center of the scene is at $x=0, y=0, z=0$. The floor is contained in (approximately) $x\in[-2.5, 2.5]$ and $y\in[-2.5, 2.5]$, with $z=0$ on the surface. 

**Q3:** Write the code that makes sphere appearing just above one of the trees and so that they then roll in various directions:

If no E-Puck is eating the spheres, it might end up with a large number of spheres occupying the environment and this could dramatically impair the V-REP performances. In that case, clean the environment by closing and restarting the session as explained at the beginning of the notebook.

## Combining behaviors

**Q4:** Define a behavior allowing the robot to catch food sources, let's call it `foraging`. The robot has to orient itself toward food sources, with a speed proportional to the proximiter activations (the closer the food source, the higher the speed) *Tip 1:* It's similar to the `aggression` behavior. *Tip 2:* Generated spheres have their names starting by "Sphere".

In [ ]:
def foraging(epuck):
    # Write your code below
    

Attach and start the behavior on the E-Puck: 

In [ ]:
# First detach previous behaviors that might still be attached to the robot
epuck.detach_all_behaviors()

# Write the code to attach and start the `foraging` behavior below



Whenever a sphere is detected by the proximeters, the robot should go towards it. However, if at one point the proximeters don't detect any sphere, the robot will probably stop (depending on how you defined the behavior). The only event that could make the robot move again would be a sphere that rolls until the proximeter detection area, what is not very likely to happen (and therefore quite a bad option if the survival of the robot depends on its foraging abilities). A solution to avoid such a blocking situation is to combine the `foraging` behavior with another one that keeps the robot in movement, as it is for example the case of the `obstacle_avoidance` behavior we have defined before. Let's attach and start the `obstacle_avoidance` behavior, but this time without detaching the previously attached `foraging` behavior:

In [ ]:
epuck.attach_behavior(obstacle_avoidance, freq=10)
epuck.start_behavior(obstacle_avoidance)

Since we haven't detach the previous behavior, the robot is now executing two behaviors in parallel. This can be checked with:

In [ ]:
epuck.check_behaviors()

which tells us that both behaviors are attached and started. In V-REP, you can see that the robot is now both foraging and avoiding obstacles. However, it often makes some weird movements, e.g. abruptly stopping and accelerating. With some imagination, one could say it looks a bit like the abrupt movements one can observe in some insects or reptiles (look e.g. at [these guys](https://www.youtube.com/watch?v=Slaz1yTpx1E)). This is due to the fact that both behaviors are sending motor commands to the robot in parallel, without any coordination between behaviors (*warning:* we are not making any assumption about how this is managed in animals here, although it is quite an interesting question). Let's for example consider the case where no sphere and no obstacle is detected by the proximeters. The `foraging` behavior will return null activation to the wheels (because this behavior returns wheel speeds proportional to the proximeter activations) and the `obstacle_avoidance` will send an activation of 1 (because this behavior returns wheel activation inversely proportional to the proximeter activations). Since both behaviors are executed in parallel, each at a frequency of 10Hz, the robot wheels will alternate between a null speed and a full speed in a random fashion, explaining why we observe those abrupt movements (as always: ask if not clear). Note that some cases are more tricky to handle. Consider for example a robot executing both a behavior for foraging and a behavior for hiding itself from predators. In that case the behaviors are conflicting against each other (the first drives the robot to expose itself while the second to hide itself).. Such a case could result in a blocking situation where the robot is unable neither to forage nor to ensure its safety. We will consider this problem of combining possibly conflicting behaviors both from the theoretical and practical points of view in the next classes.

## Floor sensor

The E-Puck is equipped with one more sensor that we haven't seen yet and that could be useful for some projects: a floor sensor. It is able to detect a line drawn on the floor if it is placed on it. Read this sensor by executing:

In [ ]:
floor_left, floor_middle, floor_right = epuck.floor_sensor()
print floor_left, floor_middle, floor_right

This returns three values, corresponding to the activity of the three floor sensors placed at the bottom of the robot: the first one on the left side, the second one on the middle side and the third one on the right side. Each of these sensors return 0.0 if no line is detected and 1.0 if a line is detected. Position the E-Puck on top of the line which is in the scene floor, so that when you execute the cell above, one of the sensors returns 1.0.(it will have to be quite precise, you can zoom in to the robot to have a better view).

**Q5:** Define a `line_following` behavior, so that when the robot is positioned on the line which is drawn on the floor, it moves forward and keeps following it. *Tip*: reduce the maximum speed of the robot to obtain a more precise behavior, for example:

In [5]:
epuck.max_speed = 1.0

def line_following(epuck):
    # Write your code below
    

Then, as usual, detach all previously attached behaviors, attach the new one and start it.

In some projects, this could be used e.g. for constraining the path followed by some robots, or for returning to a home position. Line editing is done using `Add -> Path` (check the V-REP documentation). But don't force yourself to use this feature if it doesn't bring something interesting for the project.

Only for information, the E-Puck is also equipped with a front camera. You can obtain and see the image returned by it by executing:

In [9]:
img = epuck.camera_image()

And then plot the resulting image with:

In [ ]:
# The line below is to tell the notebook to display the image just below this cell (instead of a separated window)
%pylab inline 

# And this one to plot the image:
imshow(img, origin='lower')

**However, it is not recommended to use the camera sensor for your project**, since image processing is a complicated topic which is outside the scope of this class. This is just to inform you that it is possible in case you want to play with it in another context. (By the way, if some of you want to reuse this library in the future, please contact me before because it is in constant evolution. I will provide you with the most recent version.)

## Dealing with multiple robots: preliminary instructions

This section explains rapidly how to deal with multiple robots and how to attach different behaviors to them, so that you can start working on your project. We will play with multi-robot environments in more detail in the next sessions.

First close the current session:

In [17]:
close_session(simulator, epuck)

Then restart the notebook.

Select the E-Puck robot by clicking on it in the scene. Copy-paste it, either using the `Edit` menu, or by pressing the usual editing shortcut `Ctrl-C` then `Ctrl-V`. A new E-puck will be placed in the scene at the exact same position as the previous one. Drag and drop this new robot at another position. Now you should see two robots in the scene.

Re-open a session, this time requesting the references to two E-Pucks instead of one, by executing:

In [1]:
from simulator_interface import open_session, close_session

simulator, epuck1, epuck2 = open_session(n_epucks=2)

Now you have access to the two E-Pucks through the variables `epuck1` and `epuck2` (these variables names are arbitrary, you can choose whatever you want, e.g. `predator` and `prey`). 

Now you can attach and start behaviors on each E-Puck independently, in the same way as you did before, simply using either the `epuck1` and `epuck2` variables instead of only the `epuck` one as before. I let you try by yourself, let me know if it's not clear how to proceed. 

To practice a bit and anticipate the next sessions, you can try executing both the `obstacle_avoidance` and the `aggression` behaviors on one E-Puck, and both the `obstacle_avoidance` and the `fear` behaviors on the second. Then enjoy observing a simple prey-predator interaction :)

See you next week!